In [1]:
"""
Defines one instance of a genome, which is a collection of genes and a fitness score.
 - Genes are a list of weights that are applied to each indicator to dictate whether to buy or sell.
 - Fitness is a score that is calculated by the fitness function, which is the total profit of the genome on BTC.
"""
from ta.volatility import BollingerBands
from ta.trend import ADXIndicator
from ta.momentum import RSIIndicator
import numpy as np
import pandas as pd
import random
import time


class TimerError(Exception):
    """A custom exception used to report errors in use of Timer class"""

class Timer:
    def __init__(self):
        self._start_time = None

    def start(self):
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def stop(self):
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None
        print(f"Elapsed time: {elapsed_time:0.4f} seconds")

class BuyIndicators():
	def __init__(self,data):
		self.data = data
		self.indicators = [column for column in data.columns if "buy" in column]
		self.size = len(self.indicators)

	def __str__(self):
		return "BuyIndicators: size = {}, indicators = {}".format(self.size, self.indicators)

	def print_indicators(self):
		for indicator in self.indicators:
			print(f'{indicator} has {self.data[indicator].value_counts()[1] if 1 in self.data[indicator].value_counts() else 0} buys and {self.data[indicator].value_counts()[-1] if -1 in self.data[indicator].value_counts() else 0} no buys')

	def __repr__(self):
		return self.__str__()

class SellIndicators():
	def __init__(self,data):
		self.data = data
		self.indicators = [column for column in data.columns if "sell" in column]
		self.size = len(self.indicators)

	def __str__(self):
		return "SellIndicators: size = {}, indicators = {}".format(self.size, self.indicators)
	
	def print_indicators(self):
		for indicator in self.indicators:
			print(f'{indicator} has {self.data[indicator].value_counts()[1] if 1 in self.data[indicator].value_counts() else 0} sells and {self.data[indicator].value_counts()[-1] if -1 in self.data[indicator].value_counts() else 0} no sells')

	def __repr__(self):
		return self.__str__()

class Genome():
	def __init__(self, fitness, buy_genes, sell_genes):
		self.fitness = fitness
		self.buy_genes = buy_genes
		self.sell_genes = sell_genes

	def __str__(self):
		return "Genome: fitness = {}, buy_genes = {}, sell_genes = {}".format(self.fitness, self.buy_genes, self.sell_genes)

	def __repr__(self):
		return self.__str__()

class Dataset():
	def __init__(self, data):
		self.data_name = data
		self.data = pd.read_csv("data/" +data+".csv")
		self.bollinger(5,10)
		self.bollinger(10,50)
		self.bollinger(20,50)
		self.bollinger(50,100)
		self.adx(10)
		self.adx(15)
		self.rsi(10)
		self.rsi(15)
		self.save()
		self.buy_genes = BuyIndicators(self.data).print_indicators()
		self.sell_genes = SellIndicators(self.data).print_indicators()

	
	def rsi(self, short_window=14):
		temp = RSIIndicator(close=self.data["close"], window=short_window).rsi()
		self.data[f"rsi_buy_{short_window}"] = np.where(temp < 30, 1,-1)
		self.data[f"rsi_sell_{short_window}"] = np.where(temp > 70, 1,-1)


	def bollinger(self, short_window=10, medium_window=50):
	
		short_window = short_window ### optimization target
		medium_window = medium_window ### optimization target

		indicator_bb = BollingerBands(close=self.data["close"], window=short_window, window_dev=2)
		indicator_bb_m = BollingerBands(close=self.data["close"], window=medium_window, window_dev=2)
		df_temp = pd.DataFrame()

		# Add Bollinger Bands features
		df_temp['mavg_s'] = indicator_bb.bollinger_mavg()
		df_temp['mavg_m'] = indicator_bb_m.bollinger_mavg()

		self.data[f"bollinger_buy_{short_window}_{medium_window}"] = np.where(df_temp['mavg_m'] > df_temp['mavg_s'], 1,-1)
		self.data[f"bollinger_sell_{short_window}_{medium_window}"] = np.where(df_temp['mavg_m'] < df_temp['mavg_s'], 1,-1)

	def adx(self, short_window=14):
		short_window = short_window### optimization target
		adx_indcator = ADXIndicator(high=self.data["high"],low=self.data["low"],close=self.data["close"], window=short_window)
		temp_df = pd.DataFrame(data=[adx_indcator.adx(),adx_indcator.adx_pos(),adx_indcator.adx_neg()]).transpose()
		
		temp_df["adx_buy"] = -1
		temp_df["adx_sell"] = -1
		for i in range(len(self.data)):
			if(temp_df["adx"].iloc[i-1]<25 and temp_df["adx"].iloc[i]> 25 and temp_df["adx_pos"].iloc[i] > temp_df["adx_neg"].iloc[i]):
				temp_df["adx_buy"].iloc[i] = 1

			if(temp_df["adx"].iloc[i-1]<25 and temp_df["adx"].iloc[i]> 25 and temp_df["adx_pos"].iloc[i] < temp_df["adx_neg"].iloc[i]):
				temp_df["adx_sell"].iloc[i] = 1

		self.data[f"adx_buy_{short_window}"] = temp_df["adx_buy"]
		self.data[f"adx_sell_{short_window}"] = temp_df["adx_sell"]

	def save(self):
		self.data.to_csv("data/" +self.data_name+"_processed.csv", index=False)
	
	def __str__(self):
		return "Dataset: size = {}".format(len(self.data))

	def __repr__(self):
		return self.__str__()
Dataset("ETH")

bollinger_buy_5_10 has 344 buys and 375 no buys
bollinger_buy_10_50 has 356 buys and 363 no buys
bollinger_buy_20_50 has 353 buys and 366 no buys
bollinger_buy_50_100 has 314 buys and 405 no buys
adx_buy_10 has 7 buys and 712 no buys
adx_buy_15 has 5 buys and 714 no buys
rsi_buy_10 has 45 buys and 674 no buys
rsi_buy_15 has 29 buys and 690 no buys
bollinger_sell_5_10 has 366 sells and 353 no sells
bollinger_sell_10_50 has 314 sells and 405 no sells
bollinger_sell_20_50 has 317 sells and 402 no sells
bollinger_sell_50_100 has 306 sells and 413 no sells
adx_sell_10 has 8 sells and 711 no sells
adx_sell_15 has 7 sells and 712 no sells
rsi_sell_10 has 75 sells and 644 no sells
rsi_sell_15 has 38 sells and 681 no sells


c:\Users\kaneb.DESKTOP-A94UFDN\AppData\Local\Programs\Python\Python39\lib\site-packages\ta\trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
c:\Users\kaneb.DESKTOP-A94UFDN\AppData\Local\Programs\Python\Python39\lib\site-packages\ta\trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
c:\Users\kaneb.DESKTOP-A94UFDN\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
c:\Users\kaneb.DESKTOP-A94UFDN\AppData\Local\Programs\Python\Python39\lib\site-packages\ta\trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 

Dataset: size = 719

In [2]:
import copy
from multiprocessing.pool import ThreadPool as Pool
from multiprocessing import Process

best_genome = None
class TrainGenomes():
	def __init__(self, population_size, data_name, generations):
		self.population_size = population_size
		try:
			self.data = pd.read_csv("data/" +data_name+"_processed.csv")
		except:
			print("Dataset not found, please create one!")
			quit(1)
		self.generations = generations
		self.buy_genes = BuyIndicators(self.data)
		self.sell_genes = SellIndicators(self.data)
		self.t = Timer()

	def train(self):
		global best_genome
		# Initialize the population
		population = self.initialize_population(self.population_size, self.buy_genes, self.sell_genes)
		# Evaluate the population
		for i in range(self.generations):
			self.t.start()
			population = self.evaluate_population(population)
			# Sort the population by fitness
			population.sort(key=lambda x: x.fitness, reverse=True)
			# # Keep track of the best genome
			self.save_population(population,f"pre_genomes.csv")
			print(f"Generation {i} best genome fitness: {population[0].fitness}, average fitness: {np.mean([x.fitness for x in population])}")
			best_genome = population[0]
			# Seperate elite from population
			# do tournament selection for next generation
			population = self.tournament_selection(population, 0.05,2)
			# # Add the elite to the population
			# population.extend(elite)
			population.sort(key=lambda x: x.fitness, reverse=True)
			self.save_population(population,f"post_genomes.csv")
			self.t.stop()
		# # Keep track of the number of generations with no improvement
		# no_improvement = 0
		# # Keep track of the number of generations
		# generations = 0
		# # Keep track of the time
		# start_time = time.time()
		# Keep track of the best fitness
	def save_population(self,population,name):
		#save list of population classes as csv
		lst = [x.__dict__ for x in population]
		pd.DataFrame(lst).to_csv(f"{name}.csv")


	def initialize_population(self, population_size, buy_genes, sell_genes):
		population = []
		for i in range(population_size):
			buy_genes = np.random.uniform(-1,1,buy_genes.size)
			sell_genes = np.random.uniform(-1,1,sell_genes.size)
			population.append(Genome(0, buy_genes, sell_genes))

		return population
	
	def evaluate_population(self, population):
		# pool = Pool(15)
		# try:
		# 	pool.map(self.fitness, population)
		# 	pool.close()
		# 	pool.join()
		# except KeyboardInterrupt:
		# 	print ("Caught KeyboardInterrupt, terminating workers")
		# 	pool.terminate()
		# 	pool.join()
		# 	quit()

		for genome in population:
			self.fitness(genome)
		return population
	
	def fitness(self, genome):
		buying = True
		fiat_money = 100
		btc_money = 0
		fee = 0.02 # Each buy or sell event costs 2% of current holdings.
		
		#Create a buy and sell dataframe using the genomes weights and the indicators status
		temp_df = pd.DataFrame()
		for i,indicator in enumerate(self.buy_genes.indicators):
			try:
				temp_df["buy"] += self.data[indicator] * genome.buy_genes[i]
			except KeyError:
				temp_df["buy"] = self.data[indicator] * genome.buy_genes[i]
		
		for i,indicator in enumerate(self.sell_genes.indicators):
			try:
				temp_df["sell"] += self.data[indicator] * genome.sell_genes[i]
			except KeyError:
				temp_df["sell"] = self.data[indicator] * genome.sell_genes[i]
		
		for index, row in self.data.iterrows():
			if buying:
				if temp_df.iloc[index]["buy"] > 0:
					btc_money = ( fiat_money / row['close'] ) * (1 - fee)
					fiat_money = 0
					buying = False

			elif temp_df.iloc[index]["sell"] > 0:
					fiat_money = (btc_money * row['close'] ) * (1 - fee)
					btc_money = 0
					buying = True
		genome.fitness = fiat_money + (btc_money * self.data.iloc[-1]['close'])
		return fiat_money + (btc_money * self.data.iloc[-1]['close'])
	
	def tournament_selection(self, population, elitism, tournament_size):
		# Select elite
		elite = population[:int(elitism*len(population))]
		# Initialize an empty list for the parents
		parents = []
		for _ in range(len(population) - len(elite)):
			# Randomly select tournament_size genomes
			tournament = np.random.choice(population, size=tournament_size)
			# Select the best genome
			winner = copy.deepcopy(max(tournament, key=lambda x: x.fitness))
			winner.buy_genes += np.random.uniform(-0.05, 0.05, winner.buy_genes.size)
			winner.sell_genes += np.random.uniform(-0.05, 0.05, winner.sell_genes.size)
			# Add the winner to the parents list
			parents.append(winner)
		# Add the elite to the parents list
		parents.extend(elite)
		return parents

	def mutate_population(self, population, genome_mutation_probability, gene_mutation_probability):
		# Mutate the genomes
		for genome in population:
			if np.random.uniform() < genome_mutation_probability:
				for i in range(genome.buy_genes.size):
					if np.random.uniform() < gene_mutation_probability:
						genome.buy_genes[i] += np.random.uniform(-0.05, 0.05)
				for i in range(genome.sell_genes.size):
					if np.random.uniform() < gene_mutation_probability:
						genome.sell_genes[i] += np.random.uniform(-0.05, 0.05)
	
		return population


if __name__ == "__main__":
	TrainGenomes(500, "BTC",50).train()



Generation 0 best genome fitness: 123.09864826167549, average fitness: 15.35295162008147
Elapsed time: 50.2371 seconds


In [7]:
print(best_genome)


None


In [2]:

import matplotlib.pyplot as plt
class ExamineGenome():
    def __init__(self, genome, data_name):
        self.genome = genome
        try:
            self.data = pd.read_csv("data/" +data_name+"_processed.csv")
        except:
            print("Dataset not found, please create one!")
            quit(1)
        self.buy_genes = BuyIndicators(self.data)
        self.sell_genes = SellIndicators(self.data)
        self.data["timestamp"] = pd.to_datetime(self.data['timestamp'])
    def examine(self):
        buying = True
        fiat_money = 100
        btc_money = 0
        fee = 0.02 # Each buy or sell event costs 2% of current holdings.
        #Create a buy and sell dataframe using the genomes weights and the indicators status
        temp_df = pd.DataFrame()
        buys = [[],[]]
        sells = [[],[]]
        for i,indicator in enumerate(self.buy_genes.indicators):
            try:
                temp_df["buy"] += self.data[indicator] * self.genome.buy_genes[i]
            except KeyError:
                temp_df["buy"] = self.data[indicator] * self.genome.buy_genes[i]

        for i,indicator in enumerate(self.sell_genes.indicators):
            try:
                temp_df["sell"] += self.data[indicator] * self.genome.sell_genes[i]
            except KeyError:
                temp_df["sell"] = self.data[indicator] * self.genome.sell_genes[i]

        for index, row in self.data.iterrows():
            if buying:
                if temp_df.iloc[index]["buy"] > 0:
                    buys[0].append(row['timestamp'])
                    buys[1].append(row['close'])
                    btc_money = ( fiat_money / row['close'] ) * (1 - fee)
                    fiat_money = 0
                    buying = False

            elif temp_df.iloc[index]["sell"] > 0:
                    sells[0].append(row['timestamp'])
                    sells[1].append(row['close'])
                    fiat_money = (btc_money * row['close'] ) * (1 - fee)
                    btc_money = 0
                    buying = True
        
        plt.figure(figsize=(14, 8))
        plt.title('Price and Bollinger Bands')
        plt.scatter(buys[0], buys[1], label='BUY', color='green', s=25, marker="^")
        plt.scatter(sells[0], sells[1], label='SELL', color='red', s=25, marker="v")
        plt.plot(self.data['timestamp'], self.data['close'], label='Price', alpha=0.5, color='black')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend(loc='upper left')
        plt.show()
        return fiat_money + (btc_money * self.data.iloc[-1]['close'])
print(best_genome)
ExamineGenome(best_genome, "BTC").examine()

NameError: name 'best_genome' is not defined